In [50]:
import wikipedia_helpers
import pandas as pd
import gensim.utils

In [51]:
# path to save wikipedia texts to or load from if file already existing
path = "data/komponisten_texte.json"

# create dictionary with composers as keys and \
# for each key a dictionary with the texts of the wikipedia articles \
# in the languages german, arabic, english, italian, french and spanish
data_dict = wikipedia_helpers.extract_composers_texts(path)

Texts have already been scraped to: data/komponisten_texte.json


In [52]:
df = pd.read_json(path)
# transpose index and columns of df
df = df.transpose()
df.head()

,de_title,de_text,en_title,en_text,ar_title,ar_text,fr_title,fr_text,it_title,it_text,es_title,es_text
1,Abel Ehrlich,Abel Ehrlich (* 3. September 1915 in Cranz/Ost...,Abel Ehrlich,"Abel Ehrlich (Hebrew: אבל ארליך; September 3,...",أبيل إرليخ,أبيل إرليخ (بالعبرية: אבל ארליך‏) هو ملحن إسرا...,Abel Ehrlich,"Abel Ehrlich (hébreu : אבל ארליך ; Cranz, 3 se...",,,,
2,Admiral Horthy,Miklós (Nikolaus) Horthy von Nagybánya [ˈmiklo...,Miklós Horthy,Miklós Horthy de Nagybánya (Hungarian: Vitéz n...,ميكلوش هورتي,ميكلوش هورتي (بالمجرية: Horthy Miklós) ‏(18 يو...,Miklós Horthy,L'amiral Miklós Horthy de Nagybánya (hongrois ...,Miklós Horthy,"Miklós Horthy de Nagybánya (Kenderes, 18 giugn...",Miklós Horthy,"Miklós Horthy de Nagybánya —en español, Nicolá..."
3,Alexander von Zemlinsky,"Alexander (von) Zemlinsky, Pseudonym Al Robert...",Alexander von Zemlinsky,Alexander Zemlinsky or Alexander von Zemlinsky...,ألكسندر فون زيميلنسكي,ألكسندر فون زيميلنسكي (بالألمانية: Alexander v...,Alexander von Zemlinsky,Alexander (von) Zemlinsky est un compositeur a...,Alexander von Zemlinsky,"Alexander Zemlinsky (Vienna, 14 ottobre 1871 –...",Alexander von Zemlinsky,Alexander von Zemlinsky (14 de octubre de 1872...
4,Alexandre Tansman,Alexandre Tansman (französisch Alexandré Tansm...,Alexandre Tansman,"Alexander Tansman (Polish: Aleksander Tansman,...",,,Alexandre Tansman,"Alexandre Tansman, né le 11 juin 1897 à Łódź (...",Alexandre Tansman,"Alexandre Tansman (Łódź, 12 giugno 1897 – Pari...",Alexandre Tansman,"Alexandre Tansman (Lodz, Polonia,12 de junio d..."
5,Alfons Josef Biron,,,,,,,,,,,


In [53]:
df.shape

(313, 12)

In [54]:
df.de_title

1                 Abel Ehrlich
2               Admiral Horthy
3      Alexander von Zemlinsky
4            Alexandre Tansman
5           Alfons Josef Biron
                ...           
309                           
310                           
311                           
312                           
313                           
Name: de_title, Length: 313, dtype: object

In [146]:
de_texts_processed = df.de_text.apply(gensim.utils.simple_preprocess)
en_texts_processed = df.en_text.apply(gensim.utils.simple_preprocess)
ar_texts_processed = df.ar_text.apply(gensim.utils.simple_preprocess)
fr_texts_processed = df.fr_text.apply(gensim.utils.simple_preprocess)
es_texts_processed = df.es_text.apply(gensim.utils.simple_preprocess)
it_texts_processed = df.it_text.apply(gensim.utils.simple_preprocess)


In [151]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)
# experiment with parameters, e.g. window=5

In [152]:
model.build_vocab(de_texts_processed, progress_per=10)

In [153]:
model.epochs

5

In [154]:
model.train(de_texts_processed, total_examples=model.corpus_count, epochs=model.epochs)

(973666, 1369915)

In [155]:
model.save("word2vec_de.model")

In [156]:
# find words used in similar surrounding 
model.wv.most_similar("jüdisch")

[('beide', 0.9974113702774048),
 ('ungarns', 0.9972274899482727),
 ('königreich', 0.9965260624885559),
 ('name', 0.9964842200279236),
 ('rosenthal', 0.9960684776306152),
 ('bürger', 0.9960624575614929),
 ('sogenannten', 0.9958857893943787),
 ('geborene', 0.9958823323249817),
 ('gerade', 0.9957638382911682),
 ('bip', 0.9957453608512878)]

In [158]:
model.wv.similarity(w1="pianist", w2="violinist") # Ohje...

0.97093344

In [159]:
model.wv.doesnt_match(['pianist', 'violinist', 'nazi'])

'nazi'

'jüdisch'